In [ ]:
from mt import *

In [ ]:
test()

Applied a rolling ball of radius 15 to the image before loading.

In [ ]:
# The directory containing the images
data_dir = '/Volumes/martin/200116_tandem_retina_explants/27-T3/bgsub'

# glob string for images
im_glob_dapi = os.path.join(data_dir, '200116_tandem_bAA27_T3__w1*')
im_list_dapi = sorted(glob.glob(im_glob_dapi))
im_glob_cfp = os.path.join(data_dir, '200116_tandem_bAA27_T3__w2*')
im_list_cfp = sorted(glob.glob(im_glob_cfp))
im_glob_r = os.path.join(data_dir, '200116_tandem_bAA27_T3__w3*')
im_list_r = sorted(glob.glob(im_glob_r))
im_glob_fr = os.path.join(data_dir, '200116_tandem_bAA27_T3__w4*')
im_list_fr = sorted(glob.glob(im_glob_fr))

im_list_fr

In [ ]:
i = 0
dapi_raw = skimage.io.imread(im_list_dapi[i])
cfp_raw = skimage.io.imread(im_list_cfp[i])
r_raw = skimage.io.imread(im_list_r[i])
fr_raw = skimage.io.imread(im_list_fr[i])

In [ ]:
cfp, r, fr = cfp_raw, r_raw, fr_raw

In [ ]:
cfp_adj = imadjust(cfp, lower_bound=0.1, upper_bound=99.9)
r_adj = imadjust(r, lower_bound=0.1, upper_bound=99.9)
fr_adj = imadjust(fr, lower_bound=0.1, upper_bound=99.9)

In [ ]:
bokeh.io.show(show_three_ims(cfp_adj[14], r_adj[14], fr_adj[14]))

In [ ]:
fr_adj = register_images(r_adj, fr_adj)

In [ ]:
cfp_adj_g, r_adj_g, fr_adj_g = filt_gauss(cfp_adj, r_adj, fr_adj, 3)

In [ ]:
threshold2 = threshold_otsu(cfp_adj_g)
print(threshold2, 'is where the cfp cutoff point is.')
cfp_filt_gauss_bw = cfp_adj_g > threshold2

In [ ]:
threshold3 = threshold_otsu(r_adj_g)
print(threshold3, 'is where the cfp cutoff point is.')

In [ ]:
print(threshold_multiotsu(cfp_adj_g, classes=3))

In [ ]:
print(threshold_multiotsu(cfp_adj_g, classes=4))
cfp_mo0 = cfp_adj_g > threshold_multiotsu(cfp_adj_g, classes=4)[0]
cfp_mo1 = cfp_adj_g > threshold_multiotsu(cfp_adj_g, classes=4)[2]

print(threshold_multiotsu(fr_adj_g, classes=3))
fr_mo0 = fr_adj_g > threshold_multiotsu(fr_adj_g)[0]
fr_mo1 = fr_adj_g > threshold_multiotsu(fr_adj_g)[1]

print(threshold_multiotsu(r_adj_g, classes=4))
r_mo0 = r_adj_g > threshold_multiotsu(r_adj_g, classes=4)[0]
r_mo1 = r_adj_g > threshold_multiotsu(r_adj_g, classes=4)[2]

In [ ]:
bokeh.io.show(show_four_ims(cfp_adj[14], cfp_mo0[14], cfp_filt_gauss_bw[14], cfp_mo1[14]))

The lowest threshold for CFP is the best because it captures the bright and dim nuclei as well.

In [ ]:
cfp_small = skimage.morphology.remove_small_objects(cfp_mo0, 3000)

In [ ]:
bokeh.io.show(show_three_ims(cfp_adj[13], cfp_mo0[13], cfp_small[13]))

In [ ]:
bokeh.io.show(show_six_ims(fr_adj[14], fr_mo0[14], fr_mo1[14], r_adj[14], r_mo0[14], r_mo1[14]))

The highest threshold for dots works best because it gets the brightest dots (the active sites) and NOT the single molecules.

In [ ]:
fr_small = skimage.morphology.remove_small_objects(fr_mo1, 100)
r_small = skimage.morphology.remove_small_objects(r_mo1, 100)

I tried 200 as well. It seems to remove dots that are active sites, so I will stick with 100.

In [ ]:
bokeh.io.show(show_six_ims(fr_adj[14], fr_mo1[14], fr_small[14], r_adj[14], r_mo1[14], r_small[14]))

In [ ]:
def ws(image, min_size, intensity_image):
    '''
    Function to perform watershed and remove small objects. Will generate a dataframe and output total area
    of the watershed cells.
    
    ----------
    Parameters
    ----------
    image : array
        image should be filtered and segmented
    min_size : int
        will remove all objects smaller than min_size
    intensity_image : array
        image should be raw image slice
    
    ---------
    Output
    ---------
    label_image : array
        array of labeled image after watershed, each cell has unique label
    df : DataFrame
        DataFrame of label_image with 'label', 'centroid', 'area', and 'mean_intensity'
    area : int
        sum of watershed cells 
    '''
    
    distance = ndi.distance_transform_edt(image)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((10,10,10)),
                                labels=image)
    
    selem = skimage.morphology.ball(3)
    # increased dilation of minima to prevent oversegmentation
    local_maxi_dil = skimage.morphology.dilation(local_maxi, selem)
    
    markers = ndi.label(local_maxi_dil)[0]
    labels = skimage.morphology.watershed(-distance, markers, mask=image, watershed_line=True)
    ws_mask = skimage.morphology.remove_small_objects(labels, min_size=min_size)
    label_image = skimage.measure.label(ws_mask)
    if label_image.sum() != 0:
        props = skimage.measure.regionprops_table(label_image, intensity_image=intensity_image, properties=('label',
                                                                                                 'centroid',
                                                                                                 'area',
                                                                                                 'mean_intensity'))
        df = pd.DataFrame(props)
        area = df['area'].sum()
    else:
        props = {'label': [0], 'centroid': [0], 'area': [0], 'mean_intensity': [0]}
        df = pd.DataFrame(props)
        area = df['area'].sum()
    return label_image, df, area

def ws_nuc(image, min_size, intensity_image):
    '''
    Function to perform watershed and remove small objects. Will generate a dataframe and output total area
    of the watershed cells.
    
    ----------
    Parameters
    ----------
    image : array
        image should be filtered and segmented
    min_size : int
        will remove all objects smaller than min_size
    intensity_image : array
        image should be raw image slice
    
    ---------
    Output
    ---------
    label_image : array
        array of labeled image after watershed, each cell has unique label
    df : DataFrame
        DataFrame of label_image with 'label', 'centroid', 'area', and 'mean_intensity'
    area : int
        sum of watershed cells 
    '''
    
    distance = -1 * ndi.distance_transform_edt(image)
    selem = skimage.morphology.ball(6)
    # increased selem to compensate for 0 min distance
    local_mini = skimage.morphology.h_minima(distance, 0, selem=selem)
    
    selem = skimage.morphology.ball(3)
    # increased dilation of minima to prevent oversegmentation
    local_mini_dil = skimage.morphology.dilation(local_mini, selem)
    
    mrkr = ndi.label(local_mini_dil)[0]
    labels = skimage.morphology.watershed(distance, markers = mrkr,\
                                      mask=image, watershed_line=True)
    ws_mask = skimage.morphology.remove_small_objects(labels, min_size=min_size)
    label_image = skimage.measure.label(ws_mask)
    if label_image.sum() != 0:
        props = skimage.measure.regionprops_table(label_image, intensity_image=intensity_image, properties=('label',
                                                                                                 'centroid',
                                                                                                 'area',
                                                                                                 'mean_intensity'))
        df = pd.DataFrame(props)
        area = df['area'].sum()
    else:
        props = {'label': [0], 'centroid': [0], 'area': [0], 'mean_intensity': [0]}
        df = pd.DataFrame(props)
        area = df['area'].sum()
    return label_image, df, area, distance, mrkr

In [ ]:
selem = skimage.morphology.ball(3)

# Erode binary image
cfp_bw_opened = skimage.morphology.binary_opening(cfp_small, selem)

In [ ]:
label_image_cfp, df_cfp, cfp_area, cfp_dist, cfp_mrkr = ws_nuc(cfp_bw_opened, 0, cfp)
label_image_cfp_small, df_cfp_small, cfp_area_small, cfp_dist_small, cfp_mrkr_small = ws_nuc(cfp_bw_opened, 3000, cfp)

In [ ]:
label_image_cfp_small2, df_cfp_small2, cfp_area_small2, cfp_dist_small2, cfp_mrkr_small2 = ws_nuc(cfp_bw_opened, 4000, cfp)

Tested 5000, seems too harsh, starts removing actual nuclei. 4000 seems good.

In [ ]:
i=18
bokeh.io.show(show_six_ims(cfp_adj[i], cfp_small[i], cfp_bw_opened[i], 
                           cfp_dist[i], cfp_mrkr[i], label_image_cfp[i],
                           cmap=[None, None, None, 
                                 colorcet.gray, colorcet.gray, colorcet.b_glasbey_hv]))

This is to test different min_sizes for remove_small_object.

In [ ]:
i=14
bokeh.io.show(show_six_ims(cfp_adj[i], cfp_small[i], cfp_bw_opened[i], 
                           label_image_cfp[i], label_image_cfp_small[i], label_image_cfp_small2[i],
                           cmap=[None, None, None, 
                                 colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
selem = skimage.morphology.ball(3)

# Dilate image
label_image_cfp_small_dil = skimage.morphology.dilation(label_image_cfp_small2, selem)

In [ ]:
i=14
bokeh.io.show(show_six_ims(cfp_adj[i], cfp_small[i], cfp_bw_opened[i], 
                           label_image_cfp[i], label_image_cfp_small2[i], label_image_cfp_small_dil[i],
                           cmap=[None, None, None, 
                                 colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
label_image_fr, df_fr, fr_area = ws(fr_small, 0, fr)
label_image_r, df_r, r_area = ws(r_small, 0, r)

In [ ]:
i=16
bokeh.io.show(show_six_ims(fr_adj[i], fr_mo1[i], label_image_fr[i], r_adj[i], r_mo1[i], label_image_r[i], 
                            titles=['original', 'seg', 'ws', 'original', 'seg', 'ws',], 
                             cmap=[None, None, colorcet.b_glasbey_hv, None, None, colorcet.b_glasbey_hv]))

In [ ]:
label_image_fr, df_fr, fr_area = ws(fr_small, 150, fr)
label_image_r, df_r, r_area = ws(r_small, 150, r)

In [ ]:
i=16
bokeh.io.show(show_six_ims(fr_adj[i], fr_mo1[i], label_image_fr[i], r_adj[i], r_mo1[i], label_image_r[i], 
                            titles=['original', 'seg', 'ws', 'original', 'seg', 'ws',], 
                             cmap=[None, None, colorcet.b_glasbey_hv, None, None, colorcet.b_glasbey_hv]))

In [ ]:
df_cfp_small2

In [ ]:
def freedman_diaconis_bins(data):
    """Number of bins based on Freedman-Diaconis rule."""
    h = 2 * (np.percentile(data, 75) - np.percentile(data, 25)) / np.cbrt(len(data))
    return int(np.ceil((data.max() - data.min()) / h))

bins = freedman_diaconis_bins(df_cfp_small['area'])

In [ ]:
bins

In [ ]:
hv.Histogram(
    data=np.histogram(df_cfp_small2['area'], bins=50),
    kdims=['area']
)

In [ ]:
hv.Histogram(
    data=np.histogram(df_cfp['area'], bins=50),
    kdims=['area']
)

In [ ]:
df_fr

In [ ]:
df_r

In [ ]:
labels_r = label_image_cfp_small_dil * r_mo1
labels_fr = label_image_cfp_small_dil * fr_mo1

In [ ]:
labels_r_props = skimage.measure.regionprops_table(labels_r, properties=('label','centroid'))
labels_fr_props = skimage.measure.regionprops_table(labels_fr, properties=('label','centroid'))

df_nuc_r = pd.DataFrame(labels_r_props)
df_nuc_fr = pd.DataFrame(labels_fr_props)

In [ ]:
df_nuc_r

In [ ]:
df_nuc_fr

In [ ]:
r_c0 = df_r['centroid-0'].values
r_c1 = df_r['centroid-1'].values
r_c2 = df_r['centroid-2'].values

r_centroids = np.zeros_like(r_mo1)

for i, (a, b, c) in enumerate(zip(r_c0, r_c1, r_c2)):
    r_centroids[a, b, c] = 1

In [ ]:
r_centroids.sum()

In [ ]:
fr_c0 = df_fr['centroid-0'].values
fr_c1 = df_fr['centroid-1'].values
fr_c2 = df_fr['centroid-2'].values

fr_centroids = np.zeros_like(fr_mo1)

for i, (a, b, c) in enumerate(zip(fr_c0, fr_c1, fr_c2)):
    fr_centroids[a, b, c] = 1

In [ ]:
fr_centroids.sum()

In [ ]:
r_or_fr = np.logical_or(r_centroids, fr_centroids)

In [ ]:
r_or_fr.sum()

In [ ]:
selem = skimage.morphology.ball(3)

# Dilate image
r_or_fr_dil = skimage.morphology.dilation(r_or_fr, selem)

In [ ]:
i=22
bokeh.io.show(show_four_ims(r_or_fr_dil[i], cfp_bw_opened[i],
                            label_image_cfp_small_dil[i], label_image_pos_nuc[i],
                            titles=['or', 'bw', 'watershed', 'watershed pos nuc'], 
                             cmap=[colorcet.gray, None,
                                   colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
pos_nuc = label_image_cfp_small_dil * r_or_fr

labels_pos_nuc = skimage.measure.regionprops_table(pos_nuc, properties=('label','centroid'))

df_pos_nuc = pd.DataFrame(labels_pos_nuc)

In [ ]:
df_pos_nuc

In [ ]:
label_image_pos_nuc = np.zeros_like(label_image_cfp_small_dil)

for i, label in enumerate(df_pos_nuc['label'].values):
    bool_array = label_image_cfp_small_dil == label
    single_label = bool_array * label
    label_image_pos_nuc += single_label

In [ ]:
i=14
bokeh.io.show(show_six_ims(fr_mo1[i], r_mo1[i], cfp_bw_opened[i],
                            label_image_fr[i], label_image_r[i], label_image_pos_nuc[i],
                            titles=['original', 'original', 'original', 'watershed', 'watershed', 'watershed pos nuc'], 
                             cmap=[None, None, None, 
                                   colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
list = []

for i, (a, b, c) in enumerate(zip(r_c0, r_c1, r_c2)):
    p1 = np.array([a, b, c])
    p2 = np.array([fr_c0[2], fr_c1[2], fr_c2[2]])

    squared_dist = np.sum((p1-p2)**2, axis=0)
    dist = np.sqrt(squared_dist)
    
    list.append(dist)

In [ ]:
np.asarray(list) < 4

In [ ]:
x = df_r.loc[(df_r['centroid-0'] == r_c0[0]) & 
         (df_r['centroid-1'] == r_c1[0]) & 
         (df_r['centroid-2'] == r_c2[0])]

In [ ]:
data = {'label': [], 'centroid-0': [], 'centroid-1': [], 'centroid-2': [], 'area': [], 'mean_intensity': []}
df_r_co = pd.DataFrame(data)
df_r_co.append(x)